In [1]:
import glob
import json
import os

import pandas as pd
import numpy as np

from dotenv import load_dotenv


pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 150)

Environment variables:

In [2]:
load_dotenv('envvars')

ROOT_DATA_PATH = os.environ.get('ROOT_DATA_PATH')
RAW_DATA_PATH = os.environ.get('RAW_DATA_PATH') or f'{ROOT_DATA_PATH}/raw'
CLEAN_DATA_PATH = os.environ.get('CLEAN_DATA_PATH') or f'{ROOT_DATA_PATH}/clean'

Reading data:

In [3]:
def read_csv(file, nrows=None):
    df = pd.read_csv(file, sep=';', encoding="ISO-8859-1", nrows=nrows)

    return df


files = glob.glob(f'{RAW_DATA_PATH}/*CARTERA*.csv')
dfs = [read_csv(f) for f in files]

We need to rename columns from all datasets considering there are spacing characters such as tabs and new lines in some columns' names, and there are typos that lead to different column names in multiple files containing the same variable.

In [4]:
with open(f'{ROOT_DATA_PATH}/dict-renaming-raw-columns.json', 'r') as f:
    renaming_dict = json.load(f)

for df in dfs:
    df.columns = pd.Series(df.columns).replace(renaming_dict)

df = pd.concat(dfs)
df.shape

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


(790696, 68)

In [5]:
df.head()

,BUS_REGION,CALIFICACION_CIERRE,CAPITAL_VEN,CEDULA,CELULAR,COD_LINEA,COD_MODALIDAD,COD_TIPOCLIENTE,COD_TIPO_CLIENTE,COMISION,CORREO,CUOTAS_PACTADAS,CUOTAS_PENDIENTES,DIAS_VENCIDO,DIRECCION,EDAD,EJECUTIVO_ACTUAL,ESTRATO,EST_CIVIL,FACTORRH,FECHA_UTL_ACTUALIZACION,FEC_APROBA,FEC_DESEMBOLSO,FEC_NACIMIENTO,FEC_PROXIMO_PAGO,FEC_SOLICITUD,FEC_ULT_PAGO,GARNTIA_REAL,GENERO,ID_CLIENTE,INTERES_VEN,LINEA,MODALIDAD,MONTO,MORA,MUJER_CABEZA,MUNICIPIO,MUNICIPIO_CLIENTE,NIVEL_ESTUDIO,NOMBRE,NOM_TIPOCLIENTE,NRO_SOLICITUD,OBLIGACION,OTROS,PAGARE,PERIODICIDAD,PORCENTAJE_PAGO,PROFESION,RANGO_PAGO,REGION,REGION_1,REGION_REAL,SALDO_OBLIGACION,SEGURO_VIDA,SUCURSAL,SUCURSALES,SUCURSAL_1,SUCURSAL_COD,SUCURSAL_REAL,TASA_N_A_M_V,TASA_PERIODICA,TEL_FIJO,TIPO,TIPO_CREDITO,UBICACION_CLIENTE,VALOR_CUOTA,VENCIDA,VENCIMIENTO_FINAL
0,NaN,A,-,NaN,NaN,GER,403.0,NaN,1.0,0,NaN,8.0,8.0,0.0,NaN,50.0,NaN,1.0,NaN,NaN,NaN,13/02/2017,13/02/2017,7/05/1967,13/05/2017,3/02/2017,NaN,SIN GARANTIAS REALES,Femenino,FA1303,-,GERMINA,CREDITO RURAL INDIVIDUAL - GERMINA,"2,950,000",-,N,NaN,PAZ DE ARIPORO,Primaria,NaN,Microfinanciero,1.780001e+08,1.780001e+08,0,178000070,Trimestral,######,GANADERIA,NaN,REGION NORTE,NaN,NaN,"2,950,000",-,PAZ DE ARIPORO,NaN,NaN,8,NaN,31.67,8.13,NaN,10.0,NUEVO,RURAL,520344,-,13/02/2019
1,NaN,A,-,NaN,NaN,CRE,400.0,NaN,1.0,0,NaN,12.0,10.0,0.0,NaN,64.0,NaN,NaN,NaN,NaN,NaN,24/02/2017,24/02/2017,22/03/1953,24/05/2017,20/02/2017,24/04/2017,SIN GARANTIAS REALES,Femenino,FA12443,"(1,837)",CRECER,DESARROLLO EMPRESARIAL CRECER (PYME),"1,000,000",-,Y,NaN,SABANALARGA,Primaria,NaN,Microfinanciero,1.711000e+09,1.711000e+09,0,1711000059,Mensual,85.98,SIN PROFESION,NaN,REGION SUR,NaN,NaN,"859,814",-,VILLANUEVA,NaN,NaN,4,NaN,35.29,2.94,NaN,10.0,NUEVO,URBANA,103305,-,24/02/2018
2,NaN,A,-,NaN,NaN,CRE,400.0,NaN,1.0,-6772,NaN,12.0,10.0,0.0,NaN,31.0,NaN,2.0,NaN,NaN,NaN,13/02/2017,13/02/2017,5/10/1985,13/05/2017,2/02/2017,17/04/2017,SIN GARANTIAS REALES,Masculino,FA11330,-,CRECER,DESARROLLO EMPRESARIAL CRECER (PYME),"2,900,000",-,N,NaN,YOPAL,Universitaria,NaN,Microfinanciero,1.710002e+08,1.710001e+08,0,171000109,Mensual,85.98,SIN PROFESION,NaN,REGION CENTRO,NaN,NaN,"2,493,454","(1,474)",YOPAL,NaN,NaN,1,NaN,35.29,2.94,NaN,10.0,RETANQUEADO,URBANA,299587,-,13/02/2018
3,NaN,A,(60),NaN,NaN,CRE,400.0,NaN,1.0,0,NaN,18.0,16.0,0.0,NaN,29.0,NaN,2.0,NaN,NaN,NaN,13/02/2017,13/02/2017,20/06/1987,13/05/2017,2/02/2017,12/04/2017,SIN GARANTIAS REALES,Femenino,FA19832,(123),CRECER,DESARROLLO EMPRESARIAL CRECER (PYME),"2,950,000",-,N,NaN,GRANADA,Universitaria,NaN,Microfinanciero,1.760000e+08,1.760000e+08,0,176000028,Mensual,91.61,SIN PROFESION,NaN,REGION META,NaN,NaN,"2,702,516",-,GRANADA,NaN,NaN,6,NaN,35.29,2.94,NaN,10.0,RETANQUEADO,URBANA,223070,-,13/08/2018
4,NaN,A,-,NaN,NaN,MCA,404.0,NaN,1.0,0,NaN,18.0,16.0,0.0,NaN,29.0,NaN,2.0,NaN,NaN,NaN,24/02/2017,24/02/2017,24/08/1987,24/05/2017,21/02/2017,24/04/2017,SIN GARANTIAS REALES,Masculino,FA6630,(6),MI CASA,MICROVIVIENDA,"2,000,000",-,N,NaN,VILLANUEVA,Secundaria,NaN,Microfinanciero,1.711000e+09,1.711000e+09,0,1711000061,Mensual,91.18,SIN PROFESION,NaN,REGION SUR,NaN,NaN,"1,823,637",-,VILLANUEVA,NaN,NaN,6,NaN,33.61,2.80,NaN,10.0,NUEVO,URBANA,143997,-,24/08/2018


## Draft

## SUCURSAL_COD

In [6]:
sucurs = df['SUCURSAL_COD'].replace("#N/D", np.nan).fillna(-1).astype(int).astype('category')

In [7]:
sucurs.cat.categories

Int64Index([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64')